In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, TensorDataset
import h5py
from tqdm import tqdm
from torch.amp import GradScaler, autocast
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def split_stack(arr, nrows, ncols, flatten_panels=True):
    """
    Split a stack of 2D panels into (nrows x ncols) tiles, without Python loops.

    Parameters
    ----------
    arr : ndarray, shape (P, H, W)
        P = number of panels, each of size HxW.
    nrows : int
        number of rows per tile
    ncols : int
        number of cols per tile
    flatten_panels : bool, optional (default=True)
        If True, returns shape (P * Nblocks,  nrows, ncols),
        otherwise returns (P, Nblocks, nrows, ncols).

    Returns
    -------
    tiles : ndarray
        If flatten_panels:
            shape = (P * (H//nrows)*(W//ncols),  nrows, ncols)
        else:
            shape = (P, (H//nrows)*(W//ncols),  nrows, ncols)
    """
    P, H, W = arr.shape

    #–– pad to exact multiples of nrows, ncols ––
    pad_h = (-H) % nrows
    pad_w = (-W) % ncols
    if pad_h or pad_w:
        arr = np.pad(arr,
                     ((0,0),     # no pad on panel axis
                      (0, pad_h), # pad rows
                      (0, pad_w)),# pad cols
                     mode='constant',
                     constant_values=0)

    H2, W2 = arr.shape[1], arr.shape[2]

    #–– reshape into blocks and swap axes into a block-dimension ––
    blocks = (arr
        .reshape(P,
                 H2 // nrows, nrows,
                 W2 // ncols, ncols)
        .swapaxes(2, 3)
        # now blocks.shape == (P, Hb, Wb, nrows, ncols)
    )

    #–– optionally flatten panel × block into one axis ––
    P, Hb, Wb, nr, nc = blocks.shape
    out = blocks.reshape(P * Hb * Wb, 1, nr, nc) if flatten_panels \
          else      blocks.reshape(P, Hb * Wb, nr, nc)

    return out

def prepare_train_test (data_file, train_size=0.8, seed=42):
    """    Splits the data into training and validation sets.
    Args:
        data_file (str): Path to the .npz file containing the data.
        train_size (float): Proportion of the data to use for training.
        seed (int): Random seed for reproducibility.
    Returns:
        tuple: Training and validation datasets as numpy arrays.
    """
    x,y = np.load('../DATA/train.npz').values()
    np.random.seed(seed)
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    split_idx = int(len(x) * train_size)
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]
    x_train, y_train = x[train_indices], y[train_indices]
    x_val, y_val = x[val_indices], y[val_indices]
    return x_train, y_train, x_val, y_val

def reshape_outputs(targets, img_shape=(32, 32)):
    if not isinstance(targets, torch.Tensor):
        torch_type = False
        targets = torch.tensor(targets).float()
    else:
        torch_type = True
    """
    # if shape is (..., H, W), unsqueeze to (..., 1, H, W)
    added_channel = False
    if targets.ndim >= 2 and targets.ndim < 4:
        targets = targets.unsqueeze(1)
        added_channel = True"""

    # interpolate expects shape (N, C, H, W) or (..., C, H, W)
    resized = F.interpolate(
        targets,
        size=img_shape,
        mode='bilinear',      # or 'nearest' if you prefer
        align_corners=False   # doesn't matter for nearest
    )

    # put it back to integer mask if needed
    #resized = torch.ceil(resized)
    resized = torch.round(resized)

    """if added_channel:
        resized = resized.squeeze(1)"""
    if not torch_type:
        resized = resized.numpy()
    return resized

In [ ]:
# -------------------------
# CBAM Block
# -------------------------
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, ratio=8):
        super(ChannelAttention, self).__init__()
        # We implement the shared MLP with two 1×1 convolutions.
        # First reduces channel dimension by 'ratio', then restores it.
        self.avg_pool = nn.AdaptiveAvgPool2d(1)   # output size = (B, C, 1, 1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)   # output size = (B, C, 1, 1)

        # Shared MLP: conv ↓ then conv ↑
        self.fc1 = nn.Conv2d(in_channels, in_channels // ratio, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_channels // ratio, in_channels, kernel_size=1, bias=True)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, H, W)
        b, c, _, _ = x.size()

        # 1. Average‐pooling branch
        avg_out = self.avg_pool(x)               # (B, C, 1, 1)
        avg_out = self.fc1(avg_out)              # (B, C//ratio, 1, 1)
        avg_out = self.relu(avg_out)
        avg_out = self.fc2(avg_out)              # (B, C, 1, 1)

        # 2. Max‐pooling branch
        max_out = self.max_pool(x)               # (B, C, 1, 1)
        max_out = self.fc1(max_out)              # (B, C//ratio, 1, 1)
        max_out = self.relu(max_out)
        max_out = self.fc2(max_out)              # (B, C, 1, 1)

        # 3. Combine, apply sigmoid, and scale the input
        out = avg_out + max_out                  # (B, C, 1, 1)
        scale = self.sigmoid(out)                # (B, C, 1, 1)
        return x * scale                         # broadcast along H, W


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        # kernel_size must be odd; we set padding = (kernel_size-1)//2 to preserve spatial dims
        assert kernel_size in (3, 7), "kernel size must be 3 or 7"
        padding = (kernel_size - 1) // 2

        # Convolution to produce single‐channel attention map from 2‐channel concatenation
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, H, W)
        b, c, h, w = x.size()

        # 1. Channel‐wise average: produce (B, 1, H, W)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        # 2. Channel‐wise max: also (B, 1, H, W)
        max_out, _ = torch.max(x, dim=1, keepdim=True)

        # 3. Concatenate along channel dim → (B, 2, H, W)
        concat = torch.cat([avg_out, max_out], dim=1)

        # 4. Convolution + sigmoid → spatial attention map (B, 1, H, W)
        attn = self.conv(concat)
        attn = self.sigmoid(attn)

        # 5. Scale the input feature map
        return x * attn                            # broadcast along channel dimension


class CBAM(nn.Module):
    def __init__(self, in_channels, ratio=1, kernel_size=7):
        """
        CBAM module wrapper.

        Args:
            in_channels (int): Number of input feature channels.
            ratio (int): Reduction ratio in channel attention MLP (default=1).
            kernel_size (int): Kernel size for spatial attention (either 3 or 7; default=7).
        """
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        # 1. Apply channel attention
        x_out = self.channel_attention(x)
        # 2. Apply spatial attention
        x_out = self.spatial_attention(x_out)
        return x_out

# -------------------------
# Attention Gate for Skip Connections
# -------------------------
class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1)
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1)
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, g, x):
        # g: gating signal, x: skip connection
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

# -------------------------
# Depthwise Separable Conv (for ASPP)
# -------------------------
class SepConv(nn.Module):
    def __init__(self, in_ch, out_ch, activation, kernel_size, padding, dilation=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size, padding=padding,
                                   dilation=dilation, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, 1)
        self.norm = nn.BatchNorm2d(out_ch)
        self.act = activation
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return self.act(self.norm(x))
# -------------------------
# ASPP Module with Mixed Kernels
# -------------------------
class ASPP(nn.Module):
    def __init__(self, in_ch, out_ch, activation):
        super().__init__()
        dilations = [1,1,1,1]
        kernels   = [1,3,5,7]
        self.branches = nn.ModuleList()
        for d,k in zip(dilations,kernels):
            pad = (k//2)*d
            self.branches.append(SepConv(in_ch, out_ch, activation, kernel_size=k, padding=pad, dilation=d))
        self.merge = nn.Sequential(
            nn.Conv2d(len(dilations)*out_ch, out_ch, 1),
            nn.BatchNorm2d(out_ch),
            activation,
        )
    def forward(self,x):
        return self.merge(torch.cat([b(x) for b in self.branches], dim=1))

class BottleneckTransformer(nn.Module):
    def __init__(self, dim, heads=8, depth=6, mlp_dim=None):
        super().__init__()
        mlp_dim = mlp_dim or dim*4
        layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads,
                                           dim_feedforward=mlp_dim,
                                           activation='relu', norm_first=True)
        self.encoder = nn.TransformerEncoder(layer, depth)
        self.norm    = nn.LayerNorm(dim)
    def forward(self,x):
        bs,c,h,w = x.shape
        x = x.flatten(2).permute(2,0,1)
        x = self.encoder(x)
        x = x.permute(1,2,0).view(bs,c,h,w)
        return self.norm(x.permute(0,2,3,1)).permute(0,3,1,2)

class DoubleConv(nn.Module):
    """
    A sequence of two 3x3 convolutions each followed by ReLU activation.
    """
    def __init__(self, in_channels, out_channels, activation):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            activation,
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            activation
        )

    def forward(self, x):
        return self.double_conv(x)

# -------------------------
# Encoder/Decoder Blocks
# -------------------------
class EncoderBlock(nn.Module):
    def __init__(self,in_ch,out_ch, activation, pool=True):
        super().__init__()
        #self.conv = DoubleConv(in_ch, out_ch, activation)
        self.conv = ASPP(in_ch, out_ch, activation)
        self.pool = pool
        self.cbam = CBAM(out_ch, ratio=8, kernel_size=7)
    def forward(self,x):
        x = self.conv(x)
        x = self.cbam(x)
        skip = x
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1) if self.pool else x
        return x, skip

class DecoderBlock(nn.Module):
    def __init__(self,in_ch,out_ch, activation, attn=True):
        super().__init__()
        self.up   = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.attn = attn
        self.cbam = CBAM(out_ch, ratio=8, kernel_size=7)
        if attn:
            self.attn = AttentionGate(F_g=out_ch, F_l=out_ch, F_int=out_ch//2)
            self.conv = ASPP(out_ch*2, out_ch, activation)
        else:
            self.conv = ASPP(out_ch, out_ch, activation)
    def forward(self,x,skip):
        x = self.up(x)
        if self.attn:
            skip = self.attn(g=x, x=skip)
            x = self.conv(torch.cat([x,skip],dim=1))
        else:
            x = self.conv(x)
        x = self.cbam(x)
        return x

class UNetEnhanced(nn.Module):
    """
    A basic U-Net architecture without attention, SE blocks, or deep supervision.
    """
    def __init__(self, downfeatures=None, downactivation=None, bottleneckfeatures=None, upfeatures=None, upactivation=None):
        super(UNetEnhanced, self).__init__()
        in_channels = 1
        out_channels = 1
        if downfeatures is None:
            downfeatures = [128, 256, 512, 1024]
        if bottleneckfeatures is None:
            bottleneckfeatures = downfeatures[-1] * 2
        if upfeatures is None:
            upfeatures = downfeatures[::-1]
        if downactivation is None:
            downactivation = [nn.ReLU(inplace=True) for _ in range(len(downfeatures))]
        else:
            downactivation = [get_activation(act) for act in downactivation]
        if upactivation is None:
            upactivation = downactivation[::-1]
        else:
            upactivation = [get_activation(act) for act in upactivation]

        # Encoder path
        self.downs = nn.ModuleList()
        for l, feature in enumerate(downfeatures):
            self.downs.append(EncoderBlock(in_channels, feature, downactivation[l]))
            in_channels = feature

        # Bottleneck
        self.bottleneck = DoubleConv(downfeatures[-1], bottleneckfeatures, nn.ReLU(inplace=True))

        # Decoder path
        self.ups = nn.ModuleList()
        in_channels = bottleneckfeatures
        for l, feature in enumerate(upfeatures):
            self.ups.append(DecoderBlock(in_channels, feature, upactivation[l], attn=False if l==0 else True))
            in_channels = feature

        # Final 1x1 conv
        self.final_conv = nn.Sequential(
            nn.BatchNorm2d(upfeatures[-1]),
            nn.Conv2d(upfeatures[-1], out_channels, kernel_size=3),
            nn.Sigmoid())

    def forward(self, x):
        #x = nn.LayerNorm(normalized_shape=x.shape[1:], elementwise_affine=False)(x)
        skip_connections = []

        # Downsampling
        for down in self.downs:
            x, skip = down(x)
            skip_connections.append(skip)

        # Bottleneck
        x = self.bottleneck(x)

        # Reverse skip connections for decoding
        skip_connections = skip_connections[::-1]
        for i, up in enumerate(self.ups):
            if i == 0:
                x = up(x, None)
            else:
                # Use attention gate for all but the first decoder block
                #print ("X:", x.mean(), x.std(), "SKIP:", skip_connections[i].mean(), skip_connections[i].std())
                x = up(x, skip_connections[i])
        x = self.final_conv(x)
        return x


In [ ]:
class H5Dataset(Dataset):
    def __init__(self,path): self.path=path; self.f=None
    def __len__(self): return h5py.File(self.path,'r')['x'].shape[0]
    def __getitem__(self,idx):
        if self.f is None:
            self.f = h5py.File(self.path,'r', swmr=True)
        x = torch.from_numpy(self.f['x'][idx]).float()
        y = torch.from_numpy(self.f['y'][idx]).float()
        return x, y

def get_activation(name):
    """
    Get activation function by name.
    """
    activations = {
        'relu': nn.ReLU(inplace=True),
        'leaky_relu': nn.LeakyReLU(inplace=True),
        'elu': nn.ELU(inplace=True),
        'prelu': nn.PReLU(),
        'sigmoid': nn.Sigmoid(),
        'tanh': nn.Tanh(),
        'softmax': nn.Softmax(dim=1),
    }
    return activations.get(name, None)

def train(model, train_ds, val_ds, loss=None, epochs=100, help_factor=1):
    bs, max_batches = 64, 500
     # ─── train loader ───
    # pick exactly bs*max_batches random samples from train set
    tr_idx = torch.randperm(len(train_ds))[: bs * max_batches]
    output_size = get_model_output_size(model)


    tr_loader = DataLoader(
        train_ds,
        batch_size=bs,
        sampler=SubsetRandomSampler(tr_idx),
        num_workers=10,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=8,
    )

    # ─── val loader ───
    # just iterate through val.h5 in order (or set shuffle=True if you like)
    val_loader = DataLoader(
        val_ds,
        batch_size=bs,
        shuffle=False,
        num_workers=10,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=8,
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    if loss is None:
        crit  = ComboLoss(pos_weight=500, bce_weight=0.5, dice_weight=0.5)
    else:
        crit = loss

    opt   = torch.optim.Adam(model.parameters(), lr=1e-3)
    sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-3, steps_per_epoch=len(tr_loader), epochs=epochs)

    scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')
    train_losses = []
    val_losses = []
    val_f1s = []
    val_precisions = []
    val_recalls = []

    for epoch in range(1, epochs+1):
        # ——— train ———
        model.train()
        total_loss = 0.0
        tp=fp=fn=0
        for batch_num, (imgs, msk) in enumerate(pbar:=tqdm(tr_loader, desc=f"Epoch {epoch}")):
            imgs[msk==1] *= help_factor
            imgs = imgs.to(device)
            #msk = downsize_y(msk, output_size)
            msk = reshape_outputs(msk, img_shape=output_size)
            msk  = msk.to(device)
            opt.zero_grad()
            with autocast('cuda' if torch.cuda.is_available() else 'cpu'):
                seg_logits = model(imgs)
                loss = crit(seg_logits, msk)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            scaler.step(opt)
            scaler.update()
            total_loss += loss.item()

            preds = (seg_logits>0.5).float().view(-1)
            t     = msk.view(-1)

            tp += (preds * t).sum().item()
            fp += (preds * (1-t)).sum().item()
            fn += ((1-preds)*t).sum().item()
            prec = tp/(tp+fp+1e-8)
            rec  = tp/(tp+fn+1e-8)
            f1   = 2*prec*rec/(prec+rec+1e-8)

            pbar.set_postfix(avg_train_loss=total_loss/(bs*(batch_num+1)), batch_loss=loss.item()/bs, F1=f1, prec=prec, rec=rec, lr=sched.get_last_lr()[0])

            sched.step()
        train_losses.append(total_loss/len(train_ds))

        # ——— validate ———
        model.eval()
        with torch.no_grad():
            tp=fp=fn=0
            loss = 0
            for imgs, msk in val_loader:
                imgs[msk==1] *= help_factor
                imgs = imgs.to(device)
                #msk = downsize_y(msk, output_size)
                msk = reshape_outputs(msk, img_shape=output_size)
                msk  = msk.to(device)
                with autocast('cuda' if torch.cuda.is_available() else 'cpu'):
                    seg_logits = model(imgs)
                    loss += crit(seg_logits, msk)
                preds = (seg_logits>0.5).float().view(-1)
                t     = msk.view(-1)

                tp += (preds * t).sum().item()
                fp += (preds * (1-t)).sum().item()
                fn += ((1-preds)*t).sum().item()

            prec = tp/(tp+fp+1e-8)
            rec  = tp/(tp+fn+1e-8)
            f1   = 2*prec*rec/(prec+rec+1e-8)
            val_losses.append(loss/len(val_ds))
            val_f1s.append(f1)
            val_precisions.append(prec)
            val_recalls.append(rec)
            print(f"Val loss: {loss/len(val_ds):.4f}  Val F1: {f1:.4f}  (P={prec:.4f}, R={rec:.4f})")

def downsize_y(y, size):
    """
    Downsize the y tensor to the given size.
    """
    if y.shape[-2:] == size:
        return y
    else:
        downsized = F.interpolate(y, size=size, mode='bilinear', align_corners=True)
        #downsized = F.interpolate(y, size=size, mode="area")
        # make downsized tensor only 0 or 1 by doing ceil
        #downsized = torch.ceil(downsized)
        downsized = torch.round(downsized)
        return downsized

def get_model_output_size(model):
    """
    Get the output size of the model.
    """
    x = torch.randn(1, 1, 128, 128)
    x = x.to('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        y = model(x)
    return y.shape[2:]

In [ ]:
class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.9, gamma=2.0, eps=1e-6):
        super().__init__()
        self.alpha, self.gamma, self.eps = alpha, gamma, eps
        self.beta = 1 - alpha  # Ensure alpha + beta = 1
    def forward(self, preds, targets):
        #preds = preds.view(-1)
        #targets = targets.view(-1)
        TP = (preds * targets).sum()
        FP = (preds * (1 - targets)).sum()
        FN = ((1 - preds) * targets).sum()
        tversky = (TP + self.eps) / (TP + self.alpha*FN + self.beta*FP + self.eps)
        return torch.pow((1 - tversky), self.gamma)

class ComboLoss(nn.Module):
    def __init__(self, bce_weight=0.2, dice_weight=0.4):
        super().__init__()
        self.bce  = nn.BCELoss()
        self.ft   = FocalTverskyLoss(alpha=0.995, gamma=2)
        self.bw, self.dw= bce_weight, dice_weight
    def forward(self, logits, targets):
        bce_loss  = self.bce(logits, targets)
        dice_loss = self.ft(logits, targets)
        return self.bw*bce_loss + self.dw*dice_loss

In [ ]:
def sigzi(x, axis=None):
    return 0.741 * (np.percentile(x, 75, axis=axis) - np.percentile(x, 25, axis=axis))

In [ ]:
x_train, y_train, x_val, y_val = prepare_train_test('../DATA/train.npz', train_size=0.8, seed=42)
x_train = split_stack(x_train, 128, 128)
sig = sigzi(x_train)
med = np.median(x_train)
x_train = ((x_train - med) / sig).clip(-7,7)
y_train = split_stack(y_train, 128, 128)
x_val = split_stack(x_val, 128, 128)
x_val = ((x_val - med) / sig).clip(-7,7)
y_val = split_stack(y_val, 128, 128)


down_filters = [32,  64, 64, 128, 256, 512]
up_filters   = [512, 256, 128, 64]
down_activations = ['relu', 'relu', 'relu', 'relu', 'relu', 'sigmoid', 'relu']
up_activations = ['relu', 'sigmoid', 'relu', 'relu', 'relu']
model = UNetEnhanced(downfeatures= down_filters, upfeatures=up_filters, downactivation=down_activations, upactivation=up_activations)
get_model_output_size(model)
#loss = ComboLoss(bce_weight=0, dice_weight=1)
loss = FocalTverskyLoss(alpha=0.995, gamma=2)
#train_ds = H5Dataset('train.h5')
#val_ds   = H5Dataset('val.h5')
train_ds = TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float())
val_ds   = TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).float())
#train(model, train_ds, val_ds, loss=loss, epochs=8, help_factor=100)
#train(model, train_ds, val_ds, loss=loss, epochs=8, help_factor=10)
#train(model, train_ds, val_ds, loss=loss, epochs=8, help_factor=2)
#train(model, train_ds, val_ds, loss=loss, epochs=32, help_factor=1.5)
train(model, train_ds, val_ds, loss=loss, epochs=350, help_factor=1)

In [ ]:
# make predictions with the model in batch size of 128
def predict(model, val_ds, batch_size=128):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in tqdm(range(0, len(val_ds), batch_size)):
            imgs = []
            for j in range(i, min(i+batch_size, len(val_ds))):
                img, _ = val_ds[j]
                imgs.append(img)
            imgs = torch.stack(imgs).to('cuda' if torch.cuda.is_available() else 'cpu')
            p = model(imgs)
            #p = (torch.sigmoid(p)).float()
            preds.append(p)
    return torch.cat(preds).cpu()

p = predict(model, val_ds)

In [ ]:
p.view(-1).shape, p.shape

In [ ]:
4096*32*32

In [ ]:
y_val_reduced = downsize_y(torch.from_numpy(y_val).float(), (32,32))
i = np.unique(np.argwhere(y_val==1)[:,0])[7]
fig, ax = plt.subplots(1, 3, figsize=(21, 7))
ax[0].imshow(x_val[i, 0], cmap="Grays")
ax[1].imshow((p.numpy()>=0.5)[i, 0], cmap="Grays")
ax[2].imshow(y_val_reduced[i, 0], cmap="Grays")

for a in ax:
    a.axis('off')

In [ ]:
p.numpy()[i, 0].max(), p.numpy()[i, 0].min()

In [ ]:
plt.imshow(x_val[i, 0], cmap="Grays")
plt.show()
plt.imshow((p.numpy()>=0.5)[i, 0], cmap="Grays")
plt.show()
plt.imshow(y_val_reduced[i, 0], cmap="Grays")
plt.show()

In [ ]:
y_val_reduced = downsize_y(torch.from_numpy(y_val).float(), (32,32))
index = np.unique(np.argwhere(y_val==1)[:,0])
for i in index:
    fig, ax = plt.subplots(1, 3, figsize=(21, 7))
    ax[0].imshow(x_val[i, 0], cmap="Grays")
    ax[1].imshow((p.numpy()>=0.5)[i, 0], cmap="Grays")
    ax[2].imshow(y_val_reduced[i, 0], cmap="Grays")
    for a in ax:
        a.axis('off')
    plt.show()